In [ ]:
import nd2

from ops.imports import *
from ops.process import Align
import ops.firesnake
from ops.firesnake import Snake

import ops.in_situ
import tifffile
import numpy as np
from natsort import natsorted

from matplotlib import pyplot as plt


# steps to run the analysis 
1) make sure to get the offset from **PhenotypeAlignment** notebook
2) Use the **folderPath10** as the directory to read the ISS cycle 0 files to use for alignment
3) Use the **folderPath20** as the nd2 files for phenotype (nfish) for this
4) make sure you choose the correct string identifer to look for for the filename like **'WellA1'**
5) **directory_df_10x_iter** = is the place where the analyzed barcode are kept after the ISS analysis
6) **'get_matched_df_from_phenotype'** this function is called with the offset
7) **nfish_saved_folder** = use this one where the analyzed nfish are kept (the csv files)
8) **save_matched_df**(input_20x, df_merged_with_nf, 'nfish_aligned_a1_reanalyzed_A1_3_sgrna') - use this one to save the anlyzed images
9) use **postProcessingPhenotype** to analyze the saved images
    

In [ ]:
DEBUG_PRINT = False

In [ ]:


micronPpixel10x2 = 0.643259084266003
micronPpixel20x2 = 0.1625
height10x = 2304*micronPpixel10x2
height20x = 2304*micronPpixel20x2

In [ ]:
DEBUG_PRINT

In [ ]:
#folderPath10 = '/vf/users/HiTIF/data/sagarm2/OPSData/NovData14sgRNA/ISSData/A1/20241115_114303_966_IQL_POP_043_ISS01/'
#folderPath20 = '//vf/users/HiTIF/data/sagarm2/OPSData/NovData14sgRNA/NFISH/20241114_120846_493_IQL_POP_043_NFISH/'

# dec 600 sgrna 
#folderPath10 = '/vf/users/HiTIF/data/sagarm2/OPSData/DecData600sgRNA/ISSData/20241205_132433_653_IQL_POP_044_ALTminiLib_LowDox_2days_ISS01/'
#folderPath20 = '/vf/users/HiTIF/data/sagarm2/OPSData/DecData600sgRNA/20241204_125133_089_IQL_POP_044_ALTminiLib_LowDox_2days_NFISH/'




WILDCARDS = dict(well='A1', tile='0')

#\ISSDataSet\B1\ISS01

searchRPE10=folderPath10+"/*.nd2"
input_files10 = natsorted(glob(searchRPE10))


searchRPE20=folderPath20+"/*.nd2"
input_files20_all = natsorted(glob(searchRPE20))
input_files20_list = [file_name for file_name in input_files20_all if 'WellA2' in file_name]

In [ ]:

#folderPath10 = "/vf/users/HiTIF/data/sagarm2/OPSData/NovData14sgRNA/ISSData/B1/20241115_122644_917_IQL_POP_043_ISS01_B1/"
#folderPath20 = "/vf/users/HiTIF/data/sagarm2/OPSData/NovData14sgRNA/NFISH/20241114_120846_493_IQL_POP_043_NFISH/"


#folderPath10 = "/vf/users/HiTIF/data/sagarm2/OPSData/jan2025NFISHISS_all/ISSData/A2/20250123_145415_171_IQL_POP_046_ALTminiLib_3_ISS01/"
folderPath10 = "/vf/users/HiTIF/data/sagarm2/OPSData/jan2025NFISHISS_all/ISSData/A2/20250123_145415_171_IQL_POP_046_ALTminiLib_3_ISS01/"
folderPath20 = "/vf/users/HiTIF/data/sagarm2/OPSData/jan2025NFISHISS_all/20250123_095504_119_IQL_POP_046_ALTminiLib_3_NFISH/A2/"

WILDCARDS = dict(well='A2', tile='0')
#\ISSDataSet\B1\ISS01

searchRPE10=folderPath10+"/*.nd2"
input_files10_all = natsorted(glob(searchRPE10))
input_files10 = [file_name for file_name in input_files10_all if 'WellA2' in file_name]


searchRPE20=folderPath20+"/*.nd2"
input_files20_all = natsorted(glob(searchRPE20))
input_files20_list = [file_name for file_name in input_files20_all if 'WellA2' in file_name]

print(len(input_files10), len(input_files20_list))


In [ ]:
# generate all df10 locations
# run once for single well

searchRPE1=folderPath10+"/*.nd2"
input_files1 = natsorted(glob(searchRPE1))
input_files10 = [file_name for file_name in input_files10_all if 'WellA2' in file_name]

_, micronPpixel10x2, _, _ = readPhenotypeImageToExtractLocation2(input_files1[0]) ## getting metadata infor


height10x = 2304*micronPpixel10x2
#height20x = 2304*micronPpixel20x2

data10 = []

for fileName in input_files10:
    xpos, ypos = get_position_from_nd2(fileName)
    x1_, x2_, y1_, y2_ = calculate_FOV_coord( xpos, ypos, height10x)
    data10.append((fileName, x1_, x2_, y1_, y2_))

df_loc10_all = pd.DataFrame(data10, columns=['filename','x1','x2', 'y1','y2'])



In [ ]:
input_files20_list[150]

In [ ]:
df_loc10_all.iloc[0]['filename']

In [ ]:
pwd

## offset backup
nov14sgrna A1,  xoffset=-26, yoffset=-20

Nov600sgrna, B1,  xoffset=-39, yoffset=-20


dec mini screen, A1, xoffset=44, yoffset=38

dec mini, A2, xoffset=40, yoffset=12

jan 2025
A1, xoffset=-185, yoffset=-254

A2, xoffset=-186, yoffset=-235


In [ ]:
#directory_df_10x_iter = '/gpfs/gsfs10/users/sagarm2/OPS_POP_2022_works/example_data/outputNFISHALLWELLS/B3/'
directory_df_10x_iter = '/gpfs/gsfs10/users/sagarm2/OPS_POP_2022_works/example_data/miniscreen/jan2024/A2_lowT/'
#directory_df_10x_iter = '/gpfs/gsfs10/users/sagarm2/OPS_POP_2022_works/example_data/nov14sgrna/B1/'

DEBUG_PRINT = False
#starttime = time.time()


nfish_saved_folder = 'nfish_df_A2_thresh9'

for k20 in range(0,len(input_files20_list)):
    
    input_20x = input_files20_list[k20]
    #df_merged_with_nf = get_matched_df_from_phenotype(input_20x)
    #df_merged_with_nf = get_matched_df_from_phenotype(input_20x, xoffset=3, yoffset=138 )
    #df_merged_with_nf = get_matched_df_from_phenotype(input_files20_list[k20], directory_df_10x_iter, df_loc10_all, xoffset=3, yoffset=111.4)

    try:
        df_merged_with_nf = get_matched_df_from_phenotype(input_20x, directory_df_10x_iter,df_loc10_all, nfish_saved_folder,  xoffset=-185, yoffset=-254) 
    except KeyError as e:
        print(f"KeyError encountered: {e}")
        continue
    except FileNotFoundError as e:
        print(f"FileNotFoundError encountered: {e}")
        continue
    #df_merged_with_nf = get_matched_df_from_phenotype(input_files20_list[k20], directory_df_10x_iter, df_loc10_all, xoffset=-26, yoffset=-20)
    if df_merged_with_nf.empty:
        if DEBUG_PRINT: 
            print(k20, input_20x, 'empty')
            
    else:
        #save_matched_df(input_20x, df_merged_with_nf)
        save_matched_df(input_20x, df_merged_with_nf, 'aligned_a1_lowT') # make sure this folder exits


#endTime  time.time() - starttime

In [ ]:
input_20x

In [ ]:
df_loc10_all.iloc[1]['filename']

In [ ]:
import concurrent.futures
import time

StartTime = time.time()

# make sure the nfish_df (the folder to save nfish data) is properly marked. This code reads from the nfish_df folder

#directory_df_10x_iter = '/gpfs/gsfs10/users/sagarm2/OPS_POP_2022_works/example_data/nov14sgrna/A1_3/'
directory_df_10x_iter = '/gpfs/gsfs10/users/sagarm2/OPS_POP_2022_works/example_data/miniscreen/jan2024/A2_lowT/'

nfish_saved_folder = 'nfish_df_A2_thresh9'

## try to minimize output-to-console as much as possible
## causess process pool executor to run out of memory over time
## use other methods fo
DEBUG_PRINT = False 


def process_file(input_20x):
    # Simulate your file processing
    print('Processing file:', input_20x)
    
    # Get the data for the input file
    #df_merged_with_nf = get_matched_df_from_phenotype(input_20x, directory_df_10x_iter, df_loc10_all, xoffset=3, yoffset=111.4)#B2
    #df_merged_with_nf = get_matched_df_from_phenotype(input_20x, directory_df_10x_iter,df_loc10_all, xoffset=-30, yoffset=-16) #B1
    #df_merged_with_nf = get_matched_df_from_phenotype(input_20x, directory_df_10x_iter,df_loc10_all, nfish_saved_folder, xoffset=40, yoffset=12 ) #B1
    try:
        df_merged_with_nf = get_matched_df_from_phenotype(input_20x, directory_df_10x_iter,df_loc10_all, nfish_saved_folder, xoffset=-186, yoffset=-235) #
    except KeyError as e:
        print(f"KeyError encountered: {e}")
        #continue
    except FileNotFoundError as e:
        print(f"FileNotFoundError encountered: {e}")
        #continue

    # Check if the dataframe is empty
    if df_merged_with_nf.empty:
        if DEBUG_PRINT:
            print('Empty dataframe for:', input_20x)
    else:
        # Save the matched dataframe
        save_matched_df(input_20x, df_merged_with_nf, 'aligned_a2_lowT')  # Ensure the folder exists # this is the folder wher ealigned data is saved # same path as nfish_df

# Set the maximum number of processes (for CPU-bound tasks, use processes instead of threads)
max_processes = 8  # Adjust based on your system's CPU core count

# Create a ProcessPoolExecutor for multi-processing
with concurrent.futures.ProcessPoolExecutor(max_workers=max_processes) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(process_file, input_20x) for input_20x in input_files20_list[1420:]]

    # Wait for all processes to complete
    concurrent.futures.wait(futures)

# Calculate the total time taken
timeTaken = time.time() - StartTime
print(f'Time taken: {timeTaken} seconds')


In [ ]:
len(input_files20_list)

In [ ]:
len(input_files20_list)



In [ ]:
df_merged_with_nf

In [ ]:
directory_path = '/vf/users/HiTIF/data/sagarm2/July07microNuclei/NfishDataSet/OtherNFISH/20240703_144854_601_IQL_POP_033_ALTcontrol_NFISH_B1-B3/nfish_df_b3/'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through the list of CSV files and read them into DataFrames
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames into a single large DataFrame
big_dataframe = pd.concat(dataframes, ignore_index=True)

# Optionally, inspect the resulting DataFrame
print(big_dataframe)

In [ ]:
df_filtered = big_dataframe[pd.notna(big_dataframe['cell_barcode_0'])]
len(df_filtered)

df_filtered1 = df_filtered[['cell_barcode_0','IntensitySum', 'NumberOfSpots', 'AverageSpotIntensity']]

df_filtered1 = df_filtered1.copy()
df_filtered1['GeneName'] = df_filtered1['cell_barcode_0'].map(grnaMap1_short)


In [ ]:
len(df_filtered1)

In [ ]:
def get_position_from_nd2(file_path):
    

    nd2Read = nd2.ND2File(file_path)
    
    nd22 = nd2Read.frame_metadata(0)
    positionsFg = nd22.channels[0]
    xPos = positionsFg.position.stagePositionUm.x
    yPos = positionsFg.position.stagePositionUm.y
    nd2Read.close()

    return xPos, yPos 

def readPhenotypeImageToExtractLocation(file_path_toRead):
    import nd2
    fileRead = nd2.ND2File(file_path_toRead)
    
    fg = fileRead.frame_metadata(0)
    positionsFg = fg.channels[0]
    micronPpixel = positionsFg.volume.axesCalibration[0]
    xPos = positionsFg.position.stagePositionUm.x
    yPos = positionsFg.position.stagePositionUm.y
    nd2ReadFileIm = nd2.imread(file_path_toRead)
    #print(phenotype10x.shape)
    
    phenotypeIm = np.zeros([2,2304,2304])
    phenotypeIm[0] = phenotypeIm[1] = nd2ReadFileIm #nd2ReadFile[0]
    
    fileRead.close()
    
    #matplotCreationWhole(phenotype10x[1])
    return phenotypeIm, micronPpixel, xPos, yPos


def readPhenotypeImageToExtractLocation2(file_path_toRead):
    import nd2
    fileRead = nd2.ND2File(file_path_toRead)
    
    fg = fileRead.frame_metadata(0)
    positionsFg = fg.channels[0]
    micronPpixel = positionsFg.volume.axesCalibration[0]
    xPos = positionsFg.position.stagePositionUm.x
    yPos = positionsFg.position.stagePositionUm.y
    nd2ReadFileIm = nd2.imread(file_path_toRead)
    #print(phenotype10x.shape)
    phenotypeIm = np.zeros([2,2304,2304])

    #print(nd2ReadFileIm.shape)
    if nd2ReadFileIm.ndim==2:
        phenotypeIm[0] = phenotypeIm[1] = nd2ReadFileIm #nd2ReadFile[0]
    
    elif nd2ReadFileIm.ndim == 3:
        phenotypeIm[0] = phenotypeIm[1] = nd2ReadFileIm[0] #nd2ReadFile[0]
        #raise ValueError("Unexpected image dimensions: {}".format(nd2ReadFileIm.shape))

    fileRead.close()
    
    #matplotCreationWhole(phenotype10x[1])
    return phenotypeIm, micronPpixel, xPos, yPos



def read_nd2_getXY(file_path_toRead):
    import nd2
    fileRead = nd2.ND2File(file_path_toRead)
    
    fg = fileRead.frame_metadata(0)
    positionsFg = fg.channels[0]
    micronPpixel = positionsFg.volume.axesCalibration[0]
    xPos = positionsFg.position.stagePositionUm.x
    yPos = positionsFg.position.stagePositionUm.y

    fileRead.close()
    
    #matplotCreationWhole(phenotype10x[1])
    return micronPpixel, xPos, yPos

# import cellpose


def calculate_FOV_coord(x_fov, y_fov, height):
    multiplier =0.5
    x1_ = x_fov - multiplier*height 
    x2_ = x_fov + multiplier*height 
    
    y1_ = y_fov - multiplier*height
    y2_ = y_fov + multiplier*height
    return x1_, x2_, y1_, y2_

def extract_features(phenotype_, nuclei_label):
    df_phenotype = Snake._extract_named_features(
        data = phenotype_,
        labels=nuclei_label,
        feature_names=[
            'label', # required to join SBS and phenotype data
            'i',
            'j',
            'eccentricity',
            'area',
            'solidity',
            'dapi_mean',
            'perimeter'
        ],
    
        wildcards=WILDCARDS
    )
    return df_phenotype



def find_closest(row, tree, distance_threshold=20, k=100):
    # Perform the query with a large k value to capture a sufficient number of points
    distances, indices = tree.query([[row['x_nucleus_um_global_adjusted'], row['y_nucleus_um_global_adjusted']]], k=k)
    
    # Filter the results based on the distance threshold
    filtered_results = [(dist, idx) for dist, idx in zip(distances[0], indices[0]) if dist <= distance_threshold]
    
    # Sort the results by distance
    filtered_results.sort(key=lambda x: x[0])
    
    # Extract the indices of the 5 closest points within the distance threshold
    closest_indices = [idx for _, idx in filtered_results[:5]]
    
    return closest_indices

from cellpose import models
import torch

def cellposeSegNucG(img, min_size=15):
    # Check if GPU is available and set use_gpu accordingly
    use_gpu = torch.cuda.is_available()

    model = models.Cellpose(gpu=use_gpu, model_type="nuclei")
    
    # Include the use_gpu parameter in the eval function
    res, _, _, _ = model.eval(
        img,
        channels=[0, 0],
        diameter=None,
        min_size=min_size,
        #gpu=use_gpu  # Add this line
    )
    return res
    
def cellposeSegNucGInten(img, min_size=200, intensity_threshold=None):
    # Check if GPU is available and set use_gpu accordingly
    use_gpu = torch.cuda.is_available()

    # Optionally apply an intensity threshold
    if intensity_threshold is not None:
        img = np.where(img > intensity_threshold, img, 0)

    model = models.Cellpose(gpu=use_gpu, model_type="nuclei")
    
    # Include the use_gpu parameter in the eval function
    res, _, _, _ = model.eval(
        img,
        channels=[0, 0],
        diameter=80,
        min_size=min_size
    )
    return res





multiplier =0.5
def get_20x_fov_df(input_file_k20, micronPpixel10x2, xoffset, yoffset): ## offset needs to be applied here
    
    phenotype20x2, micronPpixel20x2, xPos_20x2, yPos_20x2 = readPhenotypeImageToExtractLocation2(input_file_k20)
    NucleiCellpose20x = get_segmentation(input_file_k20)
    #NucleiCellpose20x = cellposeSegNucG(input_file_k20)
    #NucleiCellpose20x = cellposeSegNucG(phenotype20x2[0], 1000)
    
    df_phenotype_20x2 = extract_features(phenotype20x2, NucleiCellpose20x)
    height20x = 2304*micronPpixel20x2
    
    df_phenotype_20x2['x_nucleus_um'] =  height20x - df_phenotype_20x2['j']*micronPpixel20x2
    df_phenotype_20x2['y_nucleus_um'] =  df_phenotype_20x2['i']*micronPpixel20x2
    
    df_phenotype_20x2['x_nucleus_um_global'] = xPos_20x2 - multiplier*height20x +  df_phenotype_20x2['x_nucleus_um'] 
    df_phenotype_20x2['y_nucleus_um_global'] = yPos_20x2 - multiplier*height20x +  df_phenotype_20x2['y_nucleus_um'] 
    
    df2_t = df_phenotype_20x2.copy() # only do it first time
    
    uPerP10x = micronPpixel10x2
    uPerP20x = micronPpixel20x2


    #-25.8 136.8
    df2_t['x_nucleus_um_global_adjusted'] =df2_t['x_nucleus_um_global'] + xoffset
    df2_t['y_nucleus_um_global_adjusted'] =df2_t['y_nucleus_um_global'] + yoffset
    
    area_ratio = uPerP10x/uPerP20x
    df2_t['area_new'] = df2_t['area']/ (area_ratio*area_ratio)
    df2_t['perimeter_new'] = df2_t['perimeter']/ (area_ratio)
    
    df2_t['matchFoundindf1'] = -1
    df2_t['which_df1_itbelongs'] = -1
    return df2_t, uPerP20x



def get_matching_10x_fovs_backup_delete(input20file, height20x, df_loc_highMag):
    phenotype20x2, micronPpixel20x2, xPos_20x2, yPos_20x2 = readPhenotypeImageToExtractLocation2(input20file)
    
    
    x1_20, x2_20, y1_20, y2_20 = calculate_FOV_coord(xPos_20x2, yPos_20x2, height20x)
    
    overlap_results = []
    
    area_20 = (x2_20 - x1_20) * (y2_20 - y1_20)
    
    for index10, row_10 in df_loc_highMag.iterrows():
        x1_10, x2_10, y1_10, y2_10 = row_10['x1'], row_10['x2'], row_10['y1'], row_10['y2']
    
        # Calculate the overlap coordinates
        overlap_x1 = max(x1_20, x1_10)
        overlap_x2 = min(x2_20, x2_10)
        overlap_y1 = max(y1_20, y1_10)
        overlap_y2 = min(y2_20, y2_10)
    
        # Check if there is an overlap
        if overlap_x1 < overlap_x2 and overlap_y1 < overlap_y2:
            # Calculate overlap area
            overlap_area = (overlap_x2 - overlap_x1) * (overlap_y2 - overlap_y1)
            # Calculate overlap percentage with respect to the 20x view area
            overlap_percentage = (overlap_area / area_20) * 100
            overlap_results.append({
                '10x_filename': row_10['filename'],
                'OverlapPercentage': overlap_percentage,
                '10x_index': index10  # Include the index of df_loc_highMag
            })
    
    
    df_t = pd.DataFrame(overlap_results)
    return df_t


def get_matching_10x_fovs(input20file, height20x, df_loc_highMag):
    phenotype20x2, micronPpixel20x2, xPos_20x2, yPos_20x2 = readPhenotypeImageToExtractLocation2(input20file)
    
    
    x1_20, x2_20, y1_20, y2_20 = calculate_FOV_coord(xPos_20x2, yPos_20x2, height20x)
    
    overlap_results = []
    
    area_20 = (x2_20 - x1_20) * (y2_20 - y1_20)
    
    for index10, row_10 in df_loc_highMag.iterrows():
        x1_10, x2_10, y1_10, y2_10 = row_10['x1'], row_10['x2'], row_10['y1'], row_10['y2']
    
        # Calculate the overlap coordinates
        overlap_x1 = max(x1_20, x1_10)
        overlap_x2 = min(x2_20, x2_10)
        overlap_y1 = max(y1_20, y1_10)
        overlap_y2 = min(y2_20, y2_10)
    
        # Check if there is an overlap
        if overlap_x1 < overlap_x2 and overlap_y1 < overlap_y2:
            # Calculate overlap area
            overlap_area = (overlap_x2 - overlap_x1) * (overlap_y2 - overlap_y1)
            # Calculate overlap percentage with respect to the 20x view area
            overlap_percentage = (overlap_area / area_20) * 100
            overlap_results.append({
                '10x_filename': row_10['filename'],
                'OverlapPercentage': overlap_percentage,
                '10x_index': index10  # Include the index of df_loc_highMag
            })
    
    if not overlap_results:
        return pd.DataFrame()

    
    df_t = pd.DataFrame(overlap_results)
    return df_t
# function, extract 10x info
# function, extract 10x info
from scipy.spatial import cKDTree

## make this callable from a function






In [ ]:
from scipy.spatial import cKDTree

def find_match_in10x_fov_df(df_matchingrows_10x_index, input_files10, df2_, area_ratio):

    for k10 in df_matchingrows_10x_index:
        
        ## this part should go over matching 10x fovs
        phenotype10x2, micronPpixel10x2, xPos_10x2, yPos_10x2 = readPhenotypeImageToExtractLocation2(input_files10[k10])

        if DEBUG_PRINT:
            print(input_files10[k10])

        uPerP10x = micronPpixel10x2
        
        #plt.imshow(phenotype10x2[0])
        
        
        NucleiCellpose10x = get_segmentation(input_files10[k10])
        #NucleiCellpose10x = cellposeSegNucG(input_files10[k10])
        #NucleiCellpose10x = cellposeSegNucG(phenotype10x2[0], 1000)
        
        df_phenotype_10x2 = extract_features(phenotype10x2, NucleiCellpose10x)
        df_phenotype_10x2['x_nucleus_um'] =  height10x - df_phenotype_10x2['j']*micronPpixel10x2
        df_phenotype_10x2['y_nucleus_um'] =  df_phenotype_10x2['i']*micronPpixel10x2
        
        
        df_phenotype_10x2['x_nucleus_um_global'] = xPos_10x2 - multiplier*height10x + df_phenotype_10x2['x_nucleus_um'] 
        df_phenotype_10x2['y_nucleus_um_global'] = yPos_10x2 - multiplier*height10x + df_phenotype_10x2['y_nucleus_um'] 
        
        
        
        df1 = df_phenotype_10x2.copy()
        df2 = df2_.copy()
        
        df1['x_nucleus_um_global_adjusted'] = df1['x_nucleus_um_global']
        df1['y_nucleus_um_global_adjusted'] = df1['y_nucleus_um_global']

        #area_ratio = uPerP10x/uPerP20x
        df2['area_new'] = df2['area']/ (area_ratio*area_ratio)
        df2['perimeter_new'] = df2['perimeter']/ (area_ratio)
        
        points2 = df1[['x_nucleus_um_global_adjusted', 'y_nucleus_um_global_adjusted']].to_numpy()
        #points = df1[['area', 'dapi_mean']].to_numpy()
        
        # Create the KD-tree
        tree = cKDTree(points2)
        
        
        # Apply the function to each row in df2
        df2['ClosestInDf1'] = df2.apply(lambda row: find_closest(row, tree), axis=1)
        
        
        for k20 in range(len(df2)): ## go over all cell in a single FOV of a single 20x image
        
            matched_fovs = df2.iloc[k20]['ClosestInDf1']
            #print(matched_fovs)
            
            matchFound = False
            
            if len(matched_fovs) > 0:
                matchFound = False
                matchIndex = 0
                for k_10_matched_index in range(len(matched_fovs)):
                    if compare_two_cells(df2.iloc[k20], df1.iloc[df2.iloc[k20]['ClosestInDf1'][k_10_matched_index]],0.1):
                        matchFound = True
                        df2.loc[k20, 'matchFoundindf1'] = matched_fovs[k_10_matched_index]
                        df2.loc[k20, 'which_df1_itbelongs'] = k10
                        #print(k20)
                        break

    return df2, df1



import os
from skimage import io

def get_segmentation(imagePath):
    # Extract the filename without the extension
    filename = os.path.basename(imagePath)
    
    # Determine the path to the "Segmented" folder
    path_only = os.path.dirname(imagePath)
    segmented_folder = os.path.join(path_only, 'Segmented')
    
    # Construct the path to the segmented image with the .png extension
    segmentedImageName = os.path.join(segmented_folder, os.path.splitext(filename)[0] + '_segmented.png')
    
    # Read the segmented image
    segmented_DAPI = io.imread(segmentedImageName)
    
    return segmented_DAPI


def get_nfish_preanalyzed(imagePath, nfishPath):
    # Extract the filename without the extension
    filename = os.path.basename(imagePath)
    
    # Determine the path to the "Segmented" folder
    path_only = os.path.dirname(imagePath)
    segmented_folder = os.path.join(path_only, nfishPath)
    
    # Construct the path to the segmented image with the .png extension
    df_cells_nf = pd.read_csv(os.path.join(segmented_folder, os.path.splitext(filename)[0] + '_cells.csv'))
    df_spots_nf = pd.read_csv(os.path.join(segmented_folder, os.path.splitext(filename)[0] + '_spots.csv'))
        
    return df_cells_nf, df_spots_nf



## matching using morphological parameters
import math

def compare_two_cells(c20, c10, thresh_morph):
    perimeterCheck = check_within_range(c20['perimeter_new'], c10['perimeter'], thresh_morph)
    
    areaCheck = check_within_range(c20['area_new'], c10['area'], thresh_morph)
    eccenCheck = check_within_range(c20['eccentricity'], c10['eccentricity'], thresh_morph)
    solidityCheck = check_within_range(c20['solidity'], c10['solidity'], thresh_morph)

    totalCheck = perimeterCheck*1 + areaCheck*1 + eccenCheck*1 + solidityCheck*1
    #print(perimeterCheck, areaCheck,eccenCheck, solidityCheck )
    #return perimeterCheck&areaCheck&eccenCheck&solidityCheck
    return totalCheck>2
    

def check_within_range(v20,v10, thresh_morph = 0.15):
    change_val = abs(v20-v10)/v20
    return change_val < thresh_morph





def cell_match_finder(i10, df2_, df1_, thresh_morph):
    cell_in_20x = df2_.iloc[i10] # get cell form 20x FOV
    no_of_cells = len(cell_in_20x['ClosestInDf1']) # number of match found using distance

 
    found = False 
    count = -1
    for i in range(no_of_cells):
        cellID_closest = cell_in_20x['ClosestInDf1'][i]
        if DEBUG_PRINT:
            print(cellID_closest)
        cell_Closest_in_10x = df1_.iloc[cellID_closest]
        count = i
        found = compare_two_cells(cell_in_20x, cell_Closest_in_10x, thresh_morph)
        if(found):
            #print(f"match found {i} location")
            break

    

    return found, cell_in_20x['ClosestInDf1'][count] 


    


In [ ]:
def get_matched_df_from_phenotype(input_20x, directory_df_10x_iter, df_loc10_local, nfishPath, xoffset, yoffset ):
    
    #df2_b, uPerP20x = get_20x_fov_df(input_20x, micronPpixel10x2, xoffset = -25.8, yoffset = 136.8) # with offset adjustment gets the dataframe for the 20x FOC
    df2_b_f, uPerP20x = get_20x_fov_df(input_20x, micronPpixel10x2, xoffset, yoffset) # with offset adjustment gets the dataframe for the 20x FOC
    
    height10x = 2304*micronPpixel10x2
    height20x = 2304*uPerP20x
    
    # updates the dataframe df2 with matching coordinate from 10x FOV. This should be updated to load saved FOV,  now it calculates the df1 from extracting phenotype
    
    area_ratio = micronPpixel10x2/uPerP20x
    df_matchingrows = get_matching_10x_fovs(input_20x, height20x, df_loc10_local)

    if DEBUG_PRINT:
        print(df_matchingrows)




    if df_matchingrows.empty:
        if DEBUG_PRINT:
            print("The DataFrame df_matchingrows is empty.")
        return pd.DataFrame()
    else:
        if DEBUG_PRINT:
            print(df_matchingrows)
    
        #df2_calc_f, df1 = find_match_in10x_fov_df(df_matchingrows['10x_index'], input_files10, df2_b_f, area_ratio)

        df2_calc_f = find_match_in10x_fov_df_modified(df_matchingrows['10x_index'], input_files10, df2_b_f, area_ratio)
        if DEBUG_PRINT:
            print(directory_df_10x_iter)
        df_cells_read_10x_iter = get_df_cells_matching(df_matchingrows.iloc[0]['10x_index'], directory_df_10x_iter)
        
        df2_calc_f['real_cell_label'] = df2_calc_f['matchFoundindf1'].apply(lambda x: x + 1 if x > 0 else x)
                
        df_merged_f = pd.merge(df2_calc_f, df_cells_read_10x_iter, left_on='real_cell_label', right_on='cell', how='left')
    
        df_cell_nf, df_spots_nf = get_nfish_preanalyzed(input_20x, nfishPath)
        
        df_merged_with_nf = pd.merge(df_merged_f, df_cell_nf, left_on='label', right_on='CellLabel', how='left')
        return df_merged_with_nf
        


def get_df_cells_matching(positionInSeries_df_cells, directory_df_10x_iter_df_get):

    #positionInSeries = df_matchingrows.iloc[0]['10x_index']
    
    locationInFolder = 0
    #positionInSeries = df_matchingrows.iloc[0]['10x_index']
    df_cells_read_10x_iter = pd.read_csv(directory_df_10x_iter_df_get + f"df_cells_Well{locationInFolder }_Position{positionInSeries_df_cells:02}.csv")

    return df_cells_read_10x_iter




def save_matched_df(input_20x, df_cells_nf, nfish_df_directory):

    filename = os.path.basename(input_20x)
    path_only = os.path.dirname(input_20x)
    segmented_folder = os.path.join(path_only, nfish_df_directory)
    # Construct the path to the segmented image with the .png extension
    save_csv = os.path.join(segmented_folder, os.path.splitext(filename)[0] + '_cells.csv')
    if DEBUG_PRINT:
        print('file_written', save_csv)
    df_cells_nf.to_csv(save_csv)


In [ ]:
import pandas as pd
from scipy.spatial import cKDTree

def find_match_in10x_fov_df_modified(df_matchingrows_10x_index, input_files10, df2_, area_ratio):
    df2 = df2_.copy()  # Make a copy of df2 to modify

    # Loop through matching 10x FOVs
    for k10 in df_matchingrows_10x_index:
        # Work only on rows where 'which_df1_itbelongs' == -1
        df2_subset = df2.loc[df2['which_df1_itbelongs'] == -1].copy()

        # Adjust `area_new` and `perimeter_new` for df2_subset
        df2_subset.loc[:, 'area_new'] = df2_subset['area'] / (area_ratio * area_ratio)
        df2_subset.loc[:, 'perimeter_new'] = df2_subset['perimeter'] / area_ratio

        # Process each matching 10x FOV
        phenotype10x2, micronPpixel10x2, xPos_10x2, yPos_10x2 = readPhenotypeImageToExtractLocation2(input_files10[k10])
        print(f'Processing file: {input_files10[k10]}')

        # Calculate scale and coordinates
        uPerP10x = micronPpixel10x2
        NucleiCellpose10x = get_segmentation(input_files10[k10])

        # Extract features from the 10x image
        df_phenotype_10x2 = extract_features(phenotype10x2, NucleiCellpose10x)
        df_phenotype_10x2['x_nucleus_um'] = height10x - df_phenotype_10x2['j'] * micronPpixel10x2
        df_phenotype_10x2['y_nucleus_um'] = df_phenotype_10x2['i'] * micronPpixel10x2

        # Global coordinates for the 10x image nuclei
        df_phenotype_10x2['x_nucleus_um_global'] = xPos_10x2 - multiplier * height10x + df_phenotype_10x2['x_nucleus_um']
        df_phenotype_10x2['y_nucleus_um_global'] = yPos_10x2 - multiplier * height10x + df_phenotype_10x2['y_nucleus_um']

        # Adjusted global coordinates
        df_phenotype_10x2['x_nucleus_um_global_adjusted'] = df_phenotype_10x2['x_nucleus_um_global']
        df_phenotype_10x2['y_nucleus_um_global_adjusted'] = df_phenotype_10x2['y_nucleus_um_global']

        # Create a KDTree from the 10x nuclei coordinates
        points2 = df_phenotype_10x2[['x_nucleus_um_global_adjusted', 'y_nucleus_um_global_adjusted']].to_numpy()
        tree = cKDTree(points2)

        # Apply the find_closest function to each row in df2_subset
        df2_subset['ClosestInDf1'] = df2_subset.apply(lambda row: find_closest(row, tree), axis=1)

        # Transfer `ClosestInDf1` to the original `df2`
        df2.loc[df2_subset.index, 'ClosestInDf1'] = df2_subset['ClosestInDf1']

        # Loop over each row in df2_subset to find matches
        for k20 in range(len(df2_subset)):
            matched_fovs = df2_subset.iloc[k20]['ClosestInDf1']

            if len(matched_fovs) > 0:
                for k_10_matched_index in range(len(matched_fovs)):
                    if compare_two_cells(df2_subset.iloc[k20], df_phenotype_10x2.iloc[matched_fovs[k_10_matched_index]], 0.1):
                        # Only update if it has not been matched before or a new match is found
                        if df2_subset.iloc[k20]['matchFoundindf1'] == -1:
                            df2.loc[df2_subset.index[k20], 'matchFoundindf1'] = matched_fovs[k_10_matched_index]
                            df2.loc[df2_subset.index[k20], 'which_df1_itbelongs'] = k10

                        # Update the adjusted values for `area_new` and `perimeter_new`
                        df2.loc[df2_subset.index[k20], 'area_new'] = df2.loc[df2_subset.index[k20], 'area'] / (area_ratio * area_ratio)
                        df2.loc[df2_subset.index[k20], 'perimeter_new'] = df2.loc[df2_subset.index[k20], 'perimeter'] / area_ratio
                        break

        if DEBUG_PRINT:
            print(f'Matches found in this iteration: {len(df2[df2["matchFoundindf1"] != -1])}')

    return df2


In [ ]:
csv_file